# Hypothesis testing: The classical p-value based algorithm 

The null hypothesis states that there is no difference in brand awareness between the exposed and control groups in the current case.  The level of significance is set at 0.05

### Using the z-test to calculate the p-value

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
from math import ceil
%matplotlib inline

from statsmodels.stats.proportion import proportions_ztest,proportion_confint

In [2]:
df = pd.read_csv('../Data/AdSmartABdata.csv')

In [3]:
df.loc[(df['yes']==1)|(df['no']==1),'response']=1
df['response']=df['response'].fillna(0)
df.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no,response
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0,0.0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0,0.0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1,1.0
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0,0.0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0,0.0


### Choosing the sample size

The sample size is estimated using Power Analysis. This depends on the power of the test,the alpha value and the effect size

In [4]:
effect_size=sms.proportion_effectsize(0.20,0.25)
required_n=sms.NormalIndPower().solve_power(
    effect_size,
    power=0.8,
    alpha=0.05,
    ratio=1)
required_n=ceil(required_n)
required_n

1092

In [5]:
#random sampling from the dataset to abtain a sample size of 1092
control_sample=df[df['experiment']=='control'].sample(n=required_n, random_state=22)
exposed_sample=df[df['experiment']=='exposed'].sample(n=required_n, random_state=22)

ab_test=pd.concat([control_sample,exposed_sample],axis=0)
ab_test.reset_index(drop=True, inplace=True)

In [6]:
ab_test

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no,response
0,e1d49107-f02f-4e09-8750-b22ae05d841d,control,2020-07-06,6,Generic Smartphone,6,Chrome Mobile,0,0,0.0
1,8b39c872-00f5-4278-9f0f-f8a096f4d076,control,2020-07-09,6,Generic Smartphone,6,Chrome Mobile,0,0,0.0
2,6d39d727-e406-4aee-acd9-6ce280018c73,control,2020-07-04,6,Generic Smartphone,6,Chrome Mobile,0,0,0.0
3,e44b9ba2-9a29-4477-a6ee-4913293a1a8b,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0,0.0
4,9839dfb9-13ee-4361-85f6-7e132ce30396,control,2020-07-03,15,Samsung SM-G980F,6,Facebook,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...
2179,41a3b6f6-3fb1-487d-b596-00ace314417f,exposed,2020-07-10,14,Samsung SM-G900F,6,Chrome Mobile WebView,0,0,0.0
2180,ba8728e0-4cd2-4dd1-b366-1545a8628781,exposed,2020-07-06,10,Samsung SM-A105FN,6,Samsung Internet,0,0,0.0
2181,f49a6027-3cdb-4a1f-b866-ae24241858b2,exposed,2020-07-09,13,Generic Smartphone,6,Chrome Mobile,1,0,1.0
2182,9e02a887-60dc-40e0-acf6-789694c294e3,exposed,2020-07-08,15,Generic Smartphone,6,Chrome Mobile,0,0,0.0


#calculating the sample size
def sample(N,cl,e,p):
    #calculate the z-score
    z=stats.norm.ppf(1-(1-cl)/2)
    #calculate n_0 value
    n_0=z**2*p*(1-p)/e**2
    #calculate n
    n=n_0/(1+(n_0-1)/N)
    #rounding up
    n=ceil(n)
    return n
sample_size=sample(8076,0.95,0.05,0.5)
sample_size

In [7]:
conversion_rates=ab_test.groupby('experiment')['response']
#standard deviation of the proportion
std_p=lambda x: np.std(x,ddof=0)
#standard error of the proportion
se_p=lambda x:stats.sem(x,ddof=0)

conversion_rates=conversion_rates.agg([np.mean,std_p,se_p])
conversion_rates.columns=['conversion_rate','std_deviation','std_error']
conversion_rates.style.format('{:.3f}')

,conversion_rate,std_deviation,std_error
experiment,,,
control,0.148,0.355,0.011
exposed,0.160,0.367,0.011


In [8]:
control_results=ab_test[ab_test['experiment']=='control']['response']
exposed_results=ab_test[ab_test['experiment']=='exposed']['response']

n_con=control_results.count()
n_exp=exposed_results.count()
successes=[control_results.sum(),exposed_results.sum()]
nobs=[n_con, n_exp]

In [9]:
z_stat,pval=proportions_ztest(successes,nobs=nobs)
z_stat,pval

(-0.7700541596640886, 0.44126776641059784)

Since the obtained p-value is 0.44 , which is not lower than the alpha of 0.05, the null hypothesis is accepted. This means that there is no significant difference between the control group and the exposed group